# About Dataset


<hr>

## Variable Information:


**user_id:** A unique identifier assigned to each customer to track their purchases anonymously.  

**transaction_id:** A unique identifier for each transaction to ensure distinct records.  

**transaction_date:** The date when the transaction occurred, recorded in the Shamsi (Persian) calendar format.  

**transaction_time:** The exact time at which the transaction took place, recorded in HH:MM:SS format.  

**store:** The name or branch of the store where the purchase was made.  

**product_name:** The name of the product purchased in the transaction.  

**quantity:** The number of units of the product purchased.  

**unit_price:** The price per unit of the product at the time of purchase, recorded in IRR (Iranian Rials).  

**total_price:** The total cost of the product purchased, calculated as `quantity × unit_price`.  

**total_amount:** The total amount spent by the customer in the transaction, summing up all purchased items.  

**total_items:** The total number of items (across different products) purchased in the transaction.  

## 1.Data Preprocessing

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import convertdate 
import datetime
import seaborn as sns
import jdatetime
import plotly.express as px
import plotly.graph_objects as go
from khayyam import JalaliDate
from IPython.display import Markdown
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

/home/ccp/.local/lib/python3.10/site-packages/khayyam/algorithms.py:18: UserWarning: The C extension is not available. Switching to fallback python pure algorithms,so it's about 1000X slower than C implementation of the algorithms.
  warnings.warn(


In [2]:
df= pd.read_csv(r"/home/ccp/Desktop/task1/Stores_Transactions .csv")

In [3]:
df.head(5)

,user_id,transaction_id,transaction_date,transaction_time,store,product_name,quantity,unit_price,total_price,total_amount,total_items
0,1,1,1403-01-01,23:38:58,Jewelry Store,Jewelry Store Product 38,1,1658456.89,1658456.89,1658456.89,1
1,1,2,1403-01-02,16:41:29,Electronics Store,Electronics Store Product 82,1,1462060.36,1462060.36,1462060.36,1
2,1,3,1403-01-03,07:45:32,Cosmetics Store,Cosmetics Store Product 26,1,105516.10,105516.10,323033.43,2
3,1,3,1403-01-03,07:45:32,Cosmetics Store,Cosmetics Store Product 17,1,217517.33,217517.33,323033.43,2
4,1,4,1403-01-04,15:22:12,Hardware Store,Hardware Store Product 40,1,1949500.13,1949500.13,6191593.89,6


In [4]:


def get_day_of_week_persian(shamsi_date):
    # Convert string to jdatetime object
    shamsi_date = shamsi_date.replace('-', '/')
    persian_date = jdatetime.datetime.strptime(shamsi_date, "%Y/%m/%d")

    # Map weekday number (0=Saturday, 6=Friday) to Persian day names
    persian_days = ['Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']


    # Get the weekday (0=Saturday, 6=Friday)
    weekday_num = persian_date.weekday()

    # Return the Persian day of the week
    return persian_days[weekday_num]

def get_shamsi_month(shamsi_date):
    year, month, day = map(int, shamsi_date.split('-'))
    return month


def get_season(transaction_date):
    month = int(transaction_date.split("-")[1])

    # Define Persian seasons
    if month in [1, 2, 3]:   # Farvardin, Ordibehesht, Khordad
        return "Spring"
    elif month in [4, 5, 6]: # Tir, Mordad, Shahrivar
        return "Summer"
    elif month in [7, 8, 9]: # Mehr, Aban, Azar
        return "Autumn"
    else:                    # Dey, Bahman, Esfand
        return "Winter"
    

df["day_of_week"] = df["transaction_date"].apply(lambda x: get_day_of_week_persian(x))
df["hour"] = pd.to_datetime(df["transaction_time"].astype(str)).dt.hour
df["month"] = df["transaction_date"].apply(lambda x: get_shamsi_month(x))
df["season"] = df["transaction_date"].apply(lambda x: get_season(x))


# 2.EDA


In [10]:
n_rows, n_columns = df.shape
print(f"Number of columns: {n_columns} columns\nNumber of rws: {n_rows} rows")

Number of columns: 15 columns
Number of rws: 8192 rows


In [166]:
print("number of unique customers:", df['user_id'].nunique())

number of unique customers: 12


In [76]:
df.dtypes

user_id               int64
transaction_id        int64
transaction_date     object
transaction_time     object
store                object
product_name         object
quantity              int64
unit_price          float64
total_price         float64
total_amount        float64
total_items           int64
day_of_week          object
hour                  int32
month                 int64
season               object
dtype: object

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8192 entries, 0 to 8191
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           8192 non-null   int64  
 1   transaction_id    8192 non-null   int64  
 2   transaction_date  8192 non-null   object 
 3   transaction_time  8192 non-null   object 
 4   store             8192 non-null   object 
 5   product_name      8192 non-null   object 
 6   quantity          8192 non-null   int64  
 7   unit_price        8192 non-null   float64
 8   total_price       8192 non-null   float64
 9   total_amount      8192 non-null   float64
 10  total_items       8192 non-null   int64  
 11  day_of_week       8192 non-null   object 
 12  hour              8192 non-null   int32  
 13  month             8192 non-null   int64  
 14  season            8192 non-null   object 
dtypes: float64(3), int32(1), int64(5), object(6)
memory usage: 928.1+ KB


In [78]:
print('Known observations: {}\nUnique observations: {}'.format(len(df.index),len(df.drop_duplicates().index)))
if len(df.index)==len(df.drop_duplicates().index):
    print("there is no duplicate")
else:
    print("there is {} duplicates.".format(len(df.index) - len(df.drop_duplicates().index)))

Known observations: 8192
Unique observations: 8192
there is no duplicate


In [79]:
df.isna().sum()
# indices_null_tc = df[df[""].isna()].index
# display(df.iloc[indices_null_tc])

user_id             0
transaction_id      0
transaction_date    0
transaction_time    0
store               0
product_name        0
quantity            0
unit_price          0
total_price         0
total_amount        0
total_items         0
day_of_week         0
hour                0
month               0
season              0
dtype: int64

2. We don't seem to have null value, but let's check and see that we really don't have any

In [24]:
df.sort_values('transaction_id')

,user_id,transaction_id,transaction_date,transaction_time,store,product_name,quantity,unit_price,total_price,total_amount,total_items
2760,5,1,1403-01-02,07:10:17,Home Appliance Store,Home Appliance Store Product 2,2,7898447.32,15796894.64,16511554.36,3
2759,5,1,1403-01-02,07:10:17,Home Appliance Store,Home Appliance Store Product 58,1,714659.72,714659.72,16511554.36,3
3383,6,1,1403-01-01,00:13:26,Pet Store,Pet Store Product 7,4,10457.64,41830.56,94530.21,5
3384,6,1,1403-01-01,00:13:26,Pet Store,Pet Store Product 24,1,52699.65,52699.65,94530.21,5
2046,4,1,1403-01-03,20:28:37,Supermarket,Supermarket Product 140,1,8294.52,8294.52,1201159.82,26
...,...,...,...,...,...,...,...,...,...,...,...
6768,10,347,1403-07-28,16:37:01,Clothing Store,Clothing Store Product 10,4,426623.33,1706493.32,8502176.83,9
6767,10,347,1403-07-28,16:37:01,Clothing Store,Clothing Store Product 30,2,1130873.95,2261747.90,8502176.83,9
6766,10,347,1403-07-28,16:37:01,Clothing Store,Clothing Store Product 56,3,1511311.87,4533935.61,8502176.83,9
6770,10,348,1403-07-29,23:01:37,Jewelry Store,Jewelry Store Product 8,2,30701138.74,61402277.48,82651157.21,3


In [25]:
df.sort_values("transaction_date")

,user_id,transaction_id,transaction_date,transaction_time,store,product_name,quantity,unit_price,total_price,total_amount,total_items
0,1,1,1403-01-01,23:38:58,Jewelry Store,Jewelry Store Product 38,1,1658456.89,1658456.89,1658456.89,1
7534,12,2,1403-01-01,18:01:30,Pet Store,Pet Store Product 16,8,13640.58,109124.64,109124.64,8
7533,12,1,1403-01-01,09:49:53,Furniture Store,Furniture Store Product 26,3,1567981.08,4703943.24,4703943.24,3
6775,11,3,1403-01-01,23:04:39,Cosmetics Store,Cosmetics Store Product 42,1,42162.61,42162.61,42162.61,1
6774,11,2,1403-01-01,06:27:08,Home Appliance Store,Home Appliance Store Product 21,2,15492095.90,30984191.80,30984191.80,2
...,...,...,...,...,...,...,...,...,...,...,...
651,1,309,1403-07-29,16:17:25,Supermarket,Supermarket Product 89,1,26992.54,26992.54,1906417.75,36
7532,11,345,1403-07-29,22:45:21,Supermarket,Supermarket Product 11,1,29431.46,29431.46,1273301.90,30
7531,11,345,1403-07-29,22:45:21,Supermarket,Supermarket Product 32,1,93266.36,93266.36,1273301.90,30
1325,2,312,1403-07-29,02:17:36,Home Appliance Store,Home Appliance Store Product 86,2,8019184.35,16038368.70,21995964.70,3


In [151]:
df.nunique()

user_id               12
transaction_id       348
transaction_date     215
transaction_time    3762
store                 15
product_name        1446
quantity              44
unit_price          8192
total_price         8192
total_amount        3842
total_items           50
day_of_week            7
hour                  24
month                  7
season                 3
dtype: int64

In [152]:
df[df.quantity<0]

,user_id,transaction_id,transaction_date,transaction_time,store,product_name,quantity,unit_price,total_price,total_amount,total_items,day_of_week,hour,month,season


In [6]:
cat_cols = list(set(df.columns) - set(df._get_numeric_data().columns))
num_cols = list(set(df._get_numeric_data().columns))

In [83]:
cat_cols,num_cols

(['day_of_week',
  'transaction_time',
  'product_name',
  'store',
  'season',
  'transaction_date'],
 ['unit_price',
  'total_price',
  'total_amount',
  'total_items',
  'quantity',
  'month',
  'transaction_id',
  'hour',
  'user_id'])

In [19]:
cat_cols=["product_name","store"]
num_cols=["unit_price","total_price","total_amount","total_items"]
keep_cols=["user_id","transaction_id","hour","month","quantity","transaction_time","transaction_date","season","day_of_week"]

In [20]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
season_mapping = {'winter': 0, 'spring': 1, 'summer': 2, 'autumn': 3}
day_of_week_mapping = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}

df['season'] = df['season'].map(season_mapping)
df['day_of_week'] = df['day_of_week'].map(day_of_week_mapping)


scaler = StandardScaler()
scaled_data = pd.DataFrame(scaler.fit_transform(df[num_cols]), columns=num_cols)

# Apply OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' to avoid redundant dummy variables
encoded_data = pd.DataFrame(encoder.fit_transform(df[cat_cols]), columns=encoder.get_feature_names_out(cat_cols))

# Merge all data
final_data = pd.concat([df[keep_cols], scaled_data, encoded_data], axis=1)

In [21]:
final_data

,user_id,transaction_id,hour,month,quantity,transaction_time,transaction_date,season,day_of_week,unit_price,...,store_Florist Store,store_Furniture Store,store_Hardware Store,store_Home Appliance Store,store_Jewelry Store,store_Music Store,store_Pet Store,store_Sporting Goods Store,store_Supermarket,store_Toy Store
0,1,1,23,1,1,23:38:58,1403-01-01,NaN,NaN,-0.082628,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,2,16,1,1,16:41:29,1403-01-02,NaN,NaN,-0.114348,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,3,7,1,1,07:45:32,1403-01-03,NaN,NaN,-0.333443,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3,7,1,1,07:45:32,1403-01-03,NaN,NaN,-0.315354,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,4,15,1,1,15:22:12,1403-01-04,NaN,NaN,-0.035622,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,12,306,13,7,7,13:23:31,1403-07-25,NaN,NaN,-0.343275,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8188,12,307,8,7,4,08:10:55,1403-07-27,NaN,NaN,-0.338871,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8189,12,307,8,7,1,08:10:55,1403-07-27,NaN,NaN,-0.331588,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8190,12,308,22,7,9,22:01:02,1403-07-27,NaN,NaN,-0.278542,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.Descriptive Analysis

In [27]:

# Numerical columns to analyze
numerical_columns = ['quantity', 'unit_price', 'total_price', 'total_amount', 'total_items']

# Histograms to visualize distribution
for col in numerical_columns:
    fig = px.histogram(df, x=col, nbins=20, title=f'Distribution of {col}')
    fig.show()

# Boxplots to detect outliers and visualize spread
for col in numerical_columns:
    fig = px.box(df, y=col, title=f'Boxplot of {col}')
    fig.show()

# Step 2: **Breaking Down the Categories - Who is Contributing?**

# Categorical columns to analyze
categorical_columns = ['store', 'product_name']


for col in categorical_columns:
    # Aggregate the data by store and count the occurrences(counts the number of times each category appears)
    count_data = df.groupby(col).size().reset_index(name='count')  
    # Plot the bar chart with the count data
    fig = px.bar(count_data, x=col, y='count', title=f'Distribution of {col}', color=col)
    fig.update_layout(xaxis_title=col, yaxis_title='Count', xaxis_tickangle=-45)
    fig.show()

# Step 3: **Customer Behavior - What’s Driving Total Revenue?**

# Grouping by store and calculating the total revenue
store_revenue = df.groupby('store')['total_price'].sum().reset_index()
store_revenue = store_revenue.sort_values(by='total_price', ascending=False)

# Plotting total revenue by store
fig = px.bar(store_revenue, x='store', y='total_price', title='Total Revenue by Store', color='store')
fig.update_layout(xaxis_tickangle=-45, xaxis_title='Store', yaxis_title='Total Revenue')
fig.show()


# Grouping by product name and calculating total revenue
product_revenue = df.groupby('product_name')['total_price'].sum().reset_index()
product_revenue = product_revenue.sort_values(by='total_price', ascending=False)

# Plotting total revenue by product
fig = px.bar(product_revenue.head(10), x='product_name', y='total_price', title='Top 10 Products by Total Revenue', color='product_name')
fig.update_layout(xaxis_tickangle=-45, xaxis_title='Product', yaxis_title='Total Revenue')
fig.show()



In [12]:
styled_df = (
    df.describe()
    # .drop("count", axis=0)
    .drop(["user_id","transaction_id","hour","month"],axis=1)
    .style.background_gradient(axis=0, cmap="magma")
    .set_properties(**{"text-align": "center"})
    .set_table_styles([{"selector": "th", "props": [("background-color", "k")]}])
    .set_caption("Summary Statistics")
)

styled_df

,quantity,unit_price,total_price,total_amount,total_items
count,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000
mean,2.965576,2170054.747465,3426029.835074,6339662.638085,8.775513
std,3.949849,6191959.213268,9584928.924577,13930222.221079,9.698697
min,1.000000,5013.100000,5035.480000,7839.930000,1.000000
25%,1.000000,84168.182500,185300.152500,815704.240000,3.000000
50%,2.000000,287210.565000,546766.625000,1944457.820000,5.000000
75%,3.000000,935042.745000,1939030.262500,4701880.450000,10.000000
max,48.000000,49994509.860000,140641114.200000,140641114.200000,50.000000


### **1. Central Tendency (Mean & Median)**

- **Quantity**:
  - Mean = **2.97**, Median (50%) = **2.00**  
  - **interpret**: Most transactions involve **around 2 to 3 items**, but some **larger purchases pull the mean higher**.  

- **Unit Price**:
  - Mean = **2,170,054**, Median = **287,210**  
  - **interpret**: The large difference shows a **right-skewed distribution**. 

- **Total Price**:
  - Mean = **3,426,029**, Median = **546,766**  
  - **interpret**: **Some large transactions increase the mean**.  

- **Total Amount**:
  - Mean = **6,339,662**, Median = **1,944,457**  
  - **interpret**: A few **high-value purchases increase the mean**.  

- **Total Items**:
  - Mean = **8.77**, Median = **5.00**  
  - **interpret**: Most transactions involve **small orders, but some involve many items**.  

---

### **2. Spread (Standard Deviation & Range)**

- **Quantity (Std Dev = 3.95, Max = 48)**:
  - **High variability**, meaning **some orders involve much larger quantities**.  

- **Unit Price (Std Dev = 6,191,959, Max = 49,994,509)**:
  - **Huge standard deviation compared to the mean** → Some very **high-priced products affect the dataset significantly**.  

- **Total Price (Std Dev = 9,584,928, Max = 140,641,114)**:
  - **Extreme variation**, meaning a few **very large transactions dominate the dataset**.  

- **Total Amount (Std Dev = 13,930,222, Max = 140,641,114)**:
  - **Similar pattern** to total price, confirming **high-value purchases exist**.  

- **Total Items (Std Dev = 9.69, Max = 50)**:
  - Some customers buy **very large orders**, increasing the spread.  

---

### **3. Distribution (Percentiles)**

- **Quantity**:
  - **75% of orders are 3 or fewer**, but the **maximum is 48**, confirming that **most purchases are small, but a few bulk purchases exist**.  

- **Unit Price**:
  - **75% of unit prices are below 935,042**, while the **max is nearly 50M**, confirming **some extremely expensive items exist**.  

- **Total Price & Total Amount**:
  - **75% of transactions are below ~1.94M**, while the **highest go up to 140M**, meaning **a few high-value purchases dominate the dataset**.  

- **Total Items**:
  - **75% of orders involve 10 or fewer items**, while some reach **50 items**.  

---



In [9]:
df.describe(include=np.object_)

,transaction_date,transaction_time,store,product_name,day_of_week,season
count,8192,8192,8192,8192,8192,8192
unique,215,3762,15,1446,7,3
top,1403-06-06,13:24:16,Supermarket,Pet Store Product 10,Saturday,Spring
freq,70,10,973,24,1320,3562


In [32]:
df.groupby(["month","day_of_week"]).agg({"total_price":["max","min","mean"],"total_items":["count","max","mean"]}).style.background_gradient(axis=0, cmap="magma").set_properties(**{"text-align": "center"}).set_table_styles([{"selector": "th", "props": [("background-color", "k")]}])


# 3.Time-Based Pattern Analysis

In [11]:
import plotly.express as px
import plotly.graph_objects as go
transactions_per_day = df.groupby("day_of_week")["transaction_id"].nunique()

fig = px.bar(
    transactions_per_day, 
    x=transactions_per_day.index, 
    y=transactions_per_day.values, 
    title="📊 Transactions Per Day of the Week", 
    labels={"x": "Day of the Week", "y": "Number of Transactions"},
    color=transactions_per_day.values,
    color_continuous_scale="Blues",
    text=transactions_per_day.values 
)

# Highlight the days with the highest and lowest transactions
max_day = transactions_per_day.idxmax()
max_value = transactions_per_day.max()
min_day = transactions_per_day.idxmin()
min_value = transactions_per_day.min()

# Annotating the highest and lowest transaction days
fig.add_annotation(
    x=max_day, 
    y=max_value, 
    text=f"Peak: {max_day} ({max_value})", 
    showarrow=True, 
    arrowhead=2, 
    ax=0, 
    ay=-30,
    font=dict(size=12, color="white"),
    bgcolor="green",
    opacity=0.7
)

fig.add_annotation(
    x=min_day, 
    y=min_value, 
    text=f"Lowest: {min_day} ({min_value})", 
    showarrow=True, 
    arrowhead=2, 
    ax=0, 
    ay=30,
    font=dict(size=12, color="white"),
    bgcolor="red",
    opacity=0.7
)

# Add a trendline for average transactions across days of the week
avg_transactions = transactions_per_day.mean()
fig.add_trace(go.Scatter(
    x=transactions_per_day.index, 
    y=[avg_transactions] * len(transactions_per_day), 
    mode="lines", 
    name="Average Transactions",
    line=dict(color="orange", dash="dash")
))


fig.update_layout(
    title_x=0.5,  
    xaxis=dict(
        tickmode="array",
        tickvals=transactions_per_day.index,
        ticktext=[f"{day}" for day in transactions_per_day.index],
        title="Day of the Week"
    ),
    yaxis=dict(
        title="Transactions"
    ),
    hovermode="closest",  
    template="plotly_dark", 
    plot_bgcolor='rgba(0, 0, 0, 0.1)',
    margin=dict(l=40, r=40, t=60, b=40)  
)

fig.show()


In [38]:
transactions_per_hour = df.groupby("hour")["transaction_id"].nunique()

fig = px.bar(
    transactions_per_hour, 
    x=transactions_per_hour.index, 
    y=transactions_per_hour.values, 
    title="📊 Transactions per hour", 
    labels={"x": "hour", "y": "Number of Transactions"},
    color=transactions_per_hour.values,
    color_continuous_scale="Blues",
    text=transactions_per_hour.values  )

# Highlight the days with the highest and lowest transactions
max_day = transactions_per_hour.idxmax()
max_value = transactions_per_hour.max()
min_day = transactions_per_hour.idxmin()
min_value = transactions_per_hour.min()

# Annotating the highest and lowest transaction days
fig.add_annotation(
    x=max_day, 
    y=max_value, 
    text=f"Peak: {max_day} ({max_value})", 
    showarrow=True, 
    arrowhead=2, 
    ax=0, 
    ay=-30,
    font=dict(size=12, color="white"),
    bgcolor="green",
    opacity=0.7
)
fig.add_annotation(
    x=min_day, 
    y=min_value, 
    text=f"Lowest: {min_day} ({min_value})", 
    showarrow=True, 
    arrowhead=2, 
    ax=0, 
    ay=30,
    font=dict(size=12, color="white"),
    bgcolor="red",
    opacity=0.7
)

# Add a trendline for average transactions across days of the week
avg_transactions = transactions_per_hour.mean()
fig.add_trace(go.Scatter(
    x=transactions_per_hour.index, 
    y=[avg_transactions] * len(transactions_per_hour), 
    mode="lines", 
    name="Average Transactions",
    line=dict(color="orange", dash="dash")
))
fig.update_layout(
    title_x=0.5,  
    xaxis=dict(
        tickmode="array",
        tickvals=transactions_per_hour.index,
        ticktext=[f"{day}" for day in transactions_per_hour.index],
        title="Hour"
    ),
    yaxis=dict(
        title="Transactions"
    ),
    hovermode="closest", 
    template="plotly_dark", 
    plot_bgcolor='rgba(0, 0, 0, 0.1)',
    margin=dict(l=40, r=40, t=60, b=40) 
)

fig.show()

### 3.2 Analyze Monthly Spending Trends

In [132]:
import plotly.express as px
spending_per_month = df.groupby("month")["total_price"].sum()

fig = px.line(
    spending_per_month, 
    x=spending_per_month.index, 
    y=spending_per_month.values, 
    title="📈 Monthly Spending Trend", 
    labels={"x": "Month", "y": "Total Spending"},
    markers=True,
    line_shape="spline",  # Smoothing out the curve for better trend visibility
    color_discrete_sequence=["dodgerblue"]  # Consistent color to emphasize the trend
)

# Highlight the months with the highest and lowest spending
max_spending_month = spending_per_month.idxmax()
max_spending_value = spending_per_month.max()
min_spending_month = spending_per_month.idxmin()
min_spending_value = spending_per_month.min()

# Annotating the highest and lowest spending points
fig.add_annotation(
    x=max_spending_month, 
    y=max_spending_value, 
    text=f"Peak Spending: {max_spending_month} (${max_spending_value:,.2f})", 
    showarrow=True, 
    arrowhead=2, 
    ax=0, 
    ay=-30,
    font=dict(size=12, color="white"),
    bgcolor="green",
    opacity=0.7
)

fig.add_annotation(
    x=min_spending_month, 
    y=min_spending_value, 
    text=f"Lowest Spending: {min_spending_month} (${min_spending_value:,.2f})", 
    showarrow=True, 
    arrowhead=2, 
    ax=0, 
    ay=30,
    font=dict(size=12, color="white"),
    bgcolor="red",
    opacity=0.7
)

# Add a trendline to show overall spending direction
fig.add_trace(go.Scatter(
    x=spending_per_month.index, 
    y=spending_per_month.rolling(window=3).mean(),  # A  rolling average to highlight trends
    mode="lines", 
    name="Trendline",
    line=dict(color="orange", dash="dash")
))

fig.update_layout(
    title_x=0.5, 
    xaxis=dict(
        tickmode="array",
        tickvals=spending_per_month.index,
        ticktext=[f"{month}" for month in spending_per_month.index]
    ),
    yaxis=dict(
        tickprefix="$",  # Adding currency sign for clarity
        title="Total Spending"
    ),
    hovermode="x unified",  # Show all information at a specific point
    template="plotly_dark",  # Dark background for visual contrast
    plot_bgcolor='rgba(0, 0, 0, 0.1)',
    margin=dict(l=40, r=40, t=60, b=40)
)

fig.show()


### 3.4 Hourly Transactions

In [18]:
import plotly.express as px
transactions_per_hour = df.groupby("hour")["transaction_id"].nunique()
fig = px.line(
    transactions_per_hour, 
    x=transactions_per_hour.index, 
    y=transactions_per_hour.values, 
    title="⏰ Peak Shopping Hours",
    labels={"x": "Hour of the Day", "y": "Number of Transactions"},
    markers=True,
    line_shape="spline",  # Smooth curve to better represent trends
    # line_dash="solid",    # Solid line for clarity
    color_discrete_sequence=["dodgerblue"]  
)

# Highlight the peak shopping hours
peak_hour = transactions_per_hour.idxmax()  # Find the peak hour
peak_value = transactions_per_hour.max()   # Find the peak value

fig.add_annotation(
    x=peak_hour, 
    y=peak_value, 
    text=f"Peak Hour: {peak_hour}:00", 
    showarrow=True, 
    arrowhead=2, 
    ax=0, 
    ay=-40,
    font=dict(size=12, color="black"),
    bgcolor="yellow",
    opacity=0.7
)

fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=transactions_per_hour.index,  # Show every hour for clarity
        ticktext=[f"{i}:00" for i in transactions_per_hour.index],
    ),
    yaxis=dict(
        title="Transactions"
    ),
    title_x=0.5,  # Centering the title
    template="plotly_dark",  # Dark background for contrast
    plot_bgcolor='rgba(0, 0, 0, 0.1)',
    margin=dict(l=40, r=40, t=60, b=40)  # Adjusting margins for aesthetics
)

fig.show()


### 3.5 Customer Spending Trends Over Time

In [27]:

# Convert Shamsi to Gregorian
df["transaction_date"] = df["transaction_date"].apply(lambda x: jdatetime.datetime.strptime(x, "%Y-%m-%d").togregorian())
df["transaction_date"] = pd.to_datetime(df["transaction_date"])
spending_trend = df.groupby("transaction_date")["total_price"].sum()

In [ ]:
import plotly.graph_objects as go
import pandas as pd

fig = go.Figure()

# Add the actual spending data as a line with markers
fig.add_trace(go.Scatter(
    x=spending_trend.index, 
    y=spending_trend.values, 
    mode="lines+markers", 
    name="Total Spending",
    line=dict(color="royalblue", width=3),
    marker=dict(size=8, color='rgb(255, 0, 0)', symbol='circle', opacity=0.7)
))

# Add a smoothed trendline
fig.add_trace(go.Scatter(
    x=spending_trend.index, 
    y=spending_trend.rolling(window=12).mean(), 
    mode="lines", 
    name="Trendline",
    line=dict(color="green", dash='dash')
))

# Highlight significant periods
fig.add_annotation(
    x=spending_trend.index[10], 
    y=spending_trend.values[10], 
    text="Significant Event", 
    showarrow=True, 
    arrowhead=2, 
    ax=20, 
    ay=-30
)

# Update the layout
fig.update_layout(
    title="📆 Customer Spending Over Time",
    xaxis_title="Date",
    yaxis_title="Total Spending",
    hovermode="x unified",
    template="plotly_dark",
    plot_bgcolor='rgba(0,0,0,0.1)',
    xaxis=dict(tickformat="%b %Y"),
    yaxis=dict(tickprefix="$"),
    margin=dict(l=40, r=40, t=40, b=40)
)

fig.show()


In [14]:
df_grouped = df.groupby(by=['store', 'product_name'])['total_price'].sum().reset_index(name='sum')
fig = px.sunburst(df_grouped, path=['store', 'product_name'], values='sum', title='Nested Pie Chart')


fig.show()

In [18]:
df_grouped = df.groupby(by=['store', 'product_name',"transaction_date"])['store'].count().reset_index(name='count')
fig = px.sunburst(df_grouped, path=['store', 'product_name'], values='count', title='Nested Pie Chart')  
fig.show()

In [43]:
df_grouped = df.groupby(by=['store', 'product_name',"transaction_date"])['total_price'].sum().reset_index(name='sum')
fig = px.sunburst(df_grouped, path=['store', 'product_name',"transaction_date"], values='sum', title='Nested Pie Chart')  
fig.show()